### 라이브러리

In [ ]:
pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)


In [62]:
import pandas as pd
import numpy as np

import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from wordcloud import WordCloud

from konlpy.tag import Okt
from konlpy import tag
from konlpy.corpus import kolaw
from konlpy.tag import Kkma, Komoran, Okt, Hannanum #Mecab은 윈도우에서 작동 불가능

from nltk.tokenize import sent_tokenize
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

### Lyrics 불러오기

In [65]:
Lyrics = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/Lyrics_final.csv')

### 노래가사 이진분류(긍정, 부정)

In [66]:
# 행복, 희망 => 1(긍정), 슬픔, 분노 => 0(부정) 으로 이진분류

def rating_to_label(label):
    if label == 0 or label == 3:
        return 1
    else:
        return 0
    
Lyrics['y'] = Lyrics['label'].apply(lambda x: rating_to_label(x))
Lyrics_pos = Lyrics[Lyrics['y'] == 1]
Lyrics_neg = Lyrics[Lyrics['y'] == 0]
Lyrics_pos

,Lyrics,label,y
0,아침에 일어나 너에게 짧은 인사를 보낸다 아무리 멀리 떨어져 있어도 널 생각하고 있...,0,1
1,네가 맛있었다고 했던 살구빛 샐러드 맛있어서 먹고 싶어 네가 좋아했던 것들을 나에게...,0,1
2,마주치는 눈빛에 아기 아기 왠지 어색해 대체 고개를 숙여봐도 너의 발끝에 맘이 설레...,0,1
3,어깨동무까진 괜찮아 치마 손잡는 건 위험해 잘 자란 메세 진 괜찮아그래도 자냐는 메...,0,1
4,햇살이 가득 그댈 향한 가벼운 발걸음 콧노래 부르며 쇼윈도에 비친 모습을 한번 점검...,0,1
...,...,...,...
735,반쯤 미쳐가는 다 망가져 나의 삶넌 내게서 나를 지워가고 나를 부숴버려라 추락한 나...,3,1
736,무궁화 꽃이 피었습니다 빼고 다 멈춰지담 꽃이 피었습니다 현아 빼고 다 비켜 도망가...,3,1
737,아예 아예 아예 이렇게 감을 못 잡는 거야 말을 하는지 모르겠어 뭐야 싱긋싱긋 웃으...,3,1
738,나쁜 계집애 나난 나쁜 계집애 나쁜 계집애 모든 여자가 있는 고난 나난 모든 여자가...,3,1


### 긍정, 부정 감성 함수 구현

In [67]:
def LyricsWordCount(arrList, emotion):

    pos_word = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/pos_pol_word.txt', index_col = 0)
    pos_word = pd.DataFrame(pos_word).reset_index()
    pos_word.columns = ['pos_word']

    neg_word = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/odatt/neg_pol_word.txt', index_col = 0)
    neg_word = pd.DataFrame(neg_word).reset_index()
    neg_word.columns = ['neg_word']

    list = []
    i=1

    # 감성 점수 구하기
    for Lyrics in arrList:
        neg_count = 0       # 부정 점수
        pos_count = 0       # 긍정 점수
        num = 0             # 감성 점수
        
        # 긍정 점수 구하기
        for pos_text in pos_word['pos_word']:
            pos_find = Lyrics.find(pos_text)
            if(pos_find > -1):
                pos_count += 1
            else: pos_count += 0


        # 부정 점수 구하기
        for neg_text in neg_word['neg_word']:
            neg_find = Lyrics.find(neg_text)
            if(neg_find > -1):
                neg_count += 1
            else: pos_count += 0

        # 감성 점수 공식
        if emotion == '긍정':
            num = pos_count - neg_count
        elif emotion == '부정':
            num = neg_count - pos_count

        srt = f"{i}번 노래 결과 -> 긍정 : {pos_count}, 부정 : {neg_count} , 감성 점수 : {num}"
        i = i + 1

        list.append(srt)

    return list

pos_result = LyricsWordCount(Lyrics_pos['Lyrics'], "긍정")
neg_result = LyricsWordCount(Lyrics_neg['Lyrics'], "부정")

print('긍정 노래')
for pos in pos_result:
    print(pos)

print('부정 노래')
for neg in neg_result:
    print(neg)


긍정 노래
1번 노래 결과 -> 긍정 : 10, 부정 : 6 , 감성 점수 : 4
2번 노래 결과 -> 긍정 : 4, 부정 : 0 , 감성 점수 : 4
3번 노래 결과 -> 긍정 : 4, 부정 : 3 , 감성 점수 : 1
4번 노래 결과 -> 긍정 : 6, 부정 : 3 , 감성 점수 : 3
5번 노래 결과 -> 긍정 : 6, 부정 : 1 , 감성 점수 : 5
6번 노래 결과 -> 긍정 : 11, 부정 : 3 , 감성 점수 : 8
7번 노래 결과 -> 긍정 : 7, 부정 : 4 , 감성 점수 : 3
8번 노래 결과 -> 긍정 : 7, 부정 : 4 , 감성 점수 : 3
9번 노래 결과 -> 긍정 : 11, 부정 : 2 , 감성 점수 : 9
10번 노래 결과 -> 긍정 : 23, 부정 : 11 , 감성 점수 : 12
11번 노래 결과 -> 긍정 : 8, 부정 : 4 , 감성 점수 : 4
12번 노래 결과 -> 긍정 : 9, 부정 : 6 , 감성 점수 : 3
13번 노래 결과 -> 긍정 : 7, 부정 : 6 , 감성 점수 : 1
14번 노래 결과 -> 긍정 : 6, 부정 : 2 , 감성 점수 : 4
15번 노래 결과 -> 긍정 : 10, 부정 : 2 , 감성 점수 : 8
16번 노래 결과 -> 긍정 : 9, 부정 : 5 , 감성 점수 : 4
17번 노래 결과 -> 긍정 : 7, 부정 : 5 , 감성 점수 : 2
18번 노래 결과 -> 긍정 : 1, 부정 : 1 , 감성 점수 : 0
19번 노래 결과 -> 긍정 : 5, 부정 : 4 , 감성 점수 : 1
20번 노래 결과 -> 긍정 : 3, 부정 : 1 , 감성 점수 : 2
21번 노래 결과 -> 긍정 : 10, 부정 : 9 , 감성 점수 : 1
22번 노래 결과 -> 긍정 : 4, 부정 : 10 , 감성 점수 : -6
23번 노래 결과 -> 긍정 : 10, 부정 : 9 , 감성 점수 : 1
24번 노래 결과 -> 긍정 : 5, 부정 : 4 , 감성 점수 : 1
25번 노래 결과 -> 긍정 : 9, 부정 : 8 , 감성

In [73]:
# 긍정정.csv 파일 저장
import csv
with open('KUN_pos.csv', 'w', newline= '\n') as f:
  writer = csv.writer(f)
  writer.writerow(pos_result)

In [74]:
# 부정.csv 파일 저장
import csv
with open('KUN_neg.csv', 'w', newline= '\n') as f:
  writer = csv.writer(f)
  writer.writerow(neg_result)